This notebook will import helper modelues for modeling and train speech-to-text models.


In [1]:
import os
import sys
sys.path.append("../")
# Custom scripts
from scripts.generate_amharic_characters import GenerateCharacters
from scripts.metadata_loader import MetaDataLoader
from scripts.audio_generator import make_audio_gen
from scripts.model_arch import model_1, train

char_gen = GenerateCharacters()
md_loader = MetaDataLoader()

In [2]:
amharic_root = "../data/AMHARIC"

In [3]:
train_corpus_path = f"{amharic_root}/transcriptions_amharic.csv"
characters = char_gen.get_characters(md_path=train_corpus_path)
characters = sorted(characters)
characters = characters[1:]

In [4]:
char_map = {}
char_map[""] = 0
char_map["<SPACE>"] = 1
index = 2
for c in characters:
    char_map[c] = index
    index += 1
index_map = {v+1: k for k, v in char_map.items()}

In [5]:
TRAIN_CORPUS = os.path.join(amharic_root, "train_corpus.json")
VALID_CORPUS = os.path.join(amharic_root, "valid_corpus.json")

MFCC_DIM = 32
SPECTOGRAM = False
EPOCHS = 1
MODEL_NAME = "RNN_model"


MINI_BATCH_SIZE = 64

SORT_BY_DURATION=False
MAX_DURATION = 10.0

audio_gen = make_audio_gen(TRAIN_CORPUS, 
                        VALID_CORPUS, 
                        spectrogram=False, 
                        mfcc_dim=MFCC_DIM,
                        minibatch_size=MINI_BATCH_SIZE, 
                        sort_by_duration=SORT_BY_DURATION,
                        max_duration=MAX_DURATION, 
                        char_map=char_map)
# add the training data to the generator
audio_gen.load_train_data()
audio_gen.load_validation_data()

8700
2175


In [ ]:
model = model_1(input_dim=MFCC_DIM,
                units=5,
                activation='relu',
                output_dim=len(char_map)+1,
                )